# Amazon Bedrock AgentCore Runtime에서 Amazon Bedrock model을 사용하는 간단한 CrewAI Agent 호스팅

## 개요

이 튜토리얼에서는 Amazon Bedrock AgentCore Runtime을 사용하여 간단한 CrewAI agent를 호스팅하는 방법을 배웁니다. Opentelemetry instrumentation과 AWS Opentelemetry python 라이브러리를 사용하여 이 agent에 observability를 추가하고 Amazon CloudWatch GenAI Observability Dashboard에서 성능을 모니터링합니다.

### 튜토리얼 세부 정보

| 정보                | 세부 사항                                                                        |
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형       | 대화형                                                                           |
| Agent 유형          | 단일                                                                             |
| Agentic Framework   | CrewAI                                                                           |
| LLM model           | Anthropic Claude Haiku 4.5                                                     |
| 튜토리얼 구성 요소  | AgentCore Runtime에서 agent 호스팅. CrewAI 및 Amazon Bedrock Model 사용        |
| 튜토리얼 분야       | 범용                                                                             |
| 예제 복잡도         | 쉬움                                                                             |
| 사용된 SDK          | Amazon BedrockAgentCore Python SDK 및 boto3                                    |

### 튜토리얼 주요 기능

* Amazon Bedrock AgentCore Runtime에서 Agent 호스팅
* Amazon Bedrock model 사용
* CrewAI 사용
* Amazon CloudWatch GenAI Observability


### 튜토리얼 아키텍처

이 튜토리얼에서는 기존 멀티 에이전트 crew를 AgentCore runtime에 배포하는 방법을 설명합니다.

데모 목적으로 Amazon Bedrock model을 사용하는 CrewAI crew를 사용합니다.

예제에서는 웹 검색 기능을 갖춘 여행 agent를 사용합니다.
<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="60%"/>
</div>


## 사전 요구 사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Python 3.10+
* 적절한 권한이 있는 AWS 자격 증명
* Amazon Bedrock AgentCore SDK
* CrewAI
* Amazon CloudWatch 액세스
* Amazon CloudWatch에서 [transaction search](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/Enable-TransactionSearch.html) 활성화

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## AgentCore Runtime에 배포하기 위한 agent 준비

이제 agent를 AgentCore Runtime에 배포해 보겠습니다. 이를 위해 다음이 필요합니다:
* `from bedrock_agentcore.runtime import BedrockAgentCoreApp`으로 Runtime App 가져오기
* `app = BedrockAgentCoreApp()`으로 코드에서 App 초기화
* `@app.entrypoint` decorator로 호출 함수 장식
* `app.run()`으로 AgentCoreRuntime이 agent 실행을 제어하도록 설정

### Amazon Bedrock model을 사용하는 CrewAI Agent
Amazon Bedrock model을 사용하는 runtime 준비가 완료된 CrewAI Agent를 생성해 보겠습니다.

In [ ]:
%%writefile crewai_runtime_agent.py
import os

from crewai import Agent, Task, Crew, LLM
from crewai.tools import tool
from ddgs import DDGS  # DuckDuckGo 검색 라이브러리
import logging
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from opentelemetry.instrumentation.crewai import CrewAIInstrumentor

# OpenTelemetry로 CrewAI 계측 설정
# AWS OpenTelemetry distro가 자동으로 tracer provider 설정 처리
CrewAIInstrumentor().instrument()

# AgentCore Runtime 앱 초기화
app = BedrockAgentCoreApp()

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

@tool("web_search")
def web_search(query: str) -> str:
    """Search the web for current information about travel destinations, attractions, and events."""
    try:
        ddgs = DDGS()
        results = ddgs.text(query, max_results=3)
        
        formatted_results = []
        for i, result in enumerate(results, 1):
            formatted_results.append(
                f"{i}. {result.get('title', 'No title')}\n"
                f"   {result.get('body', 'No summary')}\n"
                f"   Source: {result.get('href', 'No URL')}\n"
            )
        
        return "\n".join(formatted_results) if formatted_results else "No results found."
        
    except Exception as e:
        return f"Search error: {str(e)}"

def get_llm():
    # 환경 변수에서 모델 ID와 리전 가져오기 (기본값 설정)
    model_id = os.getenv("BEDROCK_MODEL_ID", "global.anthropic.claude-haiku-4-5-20251001-v1:0")
    region = os.getenv("AWS_DEFAULT_REGION", "us-west-2")
    
    try:
        # CrewAI의 LLM 클래스로 Bedrock 모델 초기화
        llm = LLM(
            model=f"bedrock/{model_id}",
            temperature=0.7,
            max_tokens=512,
            aws_region_name=region
        )
        logger.info(f"Successfully initialized Bedrock LLM with model: {model_id} in region: {region}")
        return llm
    except Exception as e:
        logger.error(f"Failed to initialize Bedrock LLM: {str(e)}")
        logger.error("Please ensure you have proper AWS credentials configured and access to the Bedrock model")
        raise

@app.entrypoint  # AgentCore Runtime의 진입점 decorator
def crewai_agent_bedrock(payload, context):
    """
    Invoke the agent with a payload
    """
    print(f'Payload: {payload}')
    try:
        user_input = payload.get("prompt", "What are some interesting places to visit?")
        print(f"Processing request: {user_input}")
        
        llm = get_llm()

        # CrewAI Agent 생성 - 여행 추천 전문가 역할
        travel_agent = Agent(
            role='Travel Destination Researcher',
            goal='Find dream destinations matching user preferences using web search for current information',
            backstory="You are an experienced travel agent specializing in personalized travel recommendations with access to real-time web information.",
            verbose=True,
            allow_delegation=False,
            llm=llm,
            max_iter=3,  # 최대 반복 횟수 제한
            tools=[web_search]
        )

        # Agent가 수행할 Task 정의
        task = Task(
            description=f"Research and provide travel recommendations based on this request: {user_input}. Use web search to find current information about venues, events, and attractions.",
            expected_output="A comprehensive list of recommended destinations with current information, brief descriptions, and practical travel details.",
            agent=travel_agent
        )

        # Crew 생성 및 실행
        crew = Crew(
            agents=[travel_agent],
            tasks=[task],
            verbose=True
        )

        result = crew.kickoff()  # Crew 실행
        
        print("Context:\n-------\n", context)
        print("Result Raw:\n*******\n", result.raw)
        
        return {"result": result.raw}
        
    except Exception as e:
        print(f'Exception occurred: {e}')
        return {"error": f"An error occurred: {str(e)}"}

if __name__ == "__main__":
    app.run()  # AgentCore Runtime 서버 시작

## 내부에서 무슨 일이 일어나나요?

`BedrockAgentCoreApp`을 사용하면 자동으로:

* 포트 8080에서 수신 대기하는 HTTP 서버 생성
* agent의 요구 사항을 처리하기 위한 필수 `/invocations` 엔드포인트 구현
* 상태 확인을 위한 `/ping` 엔드포인트 구현 (비동기 agent에 매우 중요)
* 적절한 콘텐츠 유형 및 응답 형식 처리
* AWS 표준에 따른 오류 처리 관리

## AgentCore Runtime에 agent 배포

`CreateAgentRuntime` 작업은 포괄적인 구성 옵션을 지원하여 컨테이너 이미지, 환경 변수 및 암호화 설정을 지정할 수 있습니다. 또한 프로토콜 설정(HTTP, MCP) 및 권한 부여 메커니즘을 구성하여 클라이언트가 agent와 통신하는 방법을 제어할 수 있습니다.

**참고:** 운영 모범 사례는 코드를 컨테이너로 패키징하고 CI/CD 파이프라인 및 IaC를 사용하여 ECR에 푸시하는 것입니다.

이 튜토리얼에서는 Amazon Bedrock AgentCore Python SDK를 사용하여 아티팩트를 쉽게 패키징하고 AgentCore runtime에 배포합니다.

### AgentCore Runtime 배포 구성

다음으로 스타터 툴킷을 사용하여 entrypoint, 방금 생성한 실행 역할 및 requirements 파일로 AgentCore Runtime 배포를 구성합니다. 또한 스타터 킷이 시작 시 Amazon ECR 리포지토리를 자동으로 생성하도록 구성합니다.

구성 단계에서 애플리케이션 코드를 기반으로 docker 파일이 생성됩니다.

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

`bedrock_agentcore_starter_toolkit`을 사용하여 agent를 구성할 때 opentelemetry instrumentation을 처리한다는 점에 유의하세요.

컨테이너화된 환경(예: docker)에 대해 구성할 때 다음 명령을 추가하세요. 아래에 예제가 나와 있습니다:

`CMD ["opentelemetry-instrument", "python", "runtime_agent_main.py"]`

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "simple_crewai_travel_agent"
# AgentCore Runtime 배포 구성
response = agentcore_runtime.configure(
    entrypoint="crewai_runtime_agent.py",  # agent 진입점 파일
    auto_create_execution_role=True,  # 실행 역할 자동 생성
    auto_create_ecr=True,  # ECR 리포지토리 자동 생성
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name,
    memory_mode='NO_MEMORY'  # 메모리 비활성화
)
response

### AgentCore Runtime에 agent 시작

이제 docker 파일이 준비되었으므로 agent를 AgentCore Runtime에 시작해 보겠습니다. 이렇게 하면 Amazon ECR 리포지토리와 AgentCore Runtime이 생성됩니다.

<div style="text-align:left">
    <img src="images/launch.png" width="85%"/>
</div>

In [ ]:
# CrewAI의 내장 telemetry 비활성화 (충돌 방지)
launch_result = agentcore_runtime.launch(env_vars={
        "CREWAI_DISABLE_TELEMETRY": "true",
        "OTEL_PYTHON_EXCLUDED_URLS": "https://api.scarf.sh/"  # OpenTelemetry에서 제외할 URL
    })
launch_result

### AgentCore Runtime 상태 확인
이제 AgentCore Runtime을 배포했으므로 배포 상태를 확인해 보겠습니다.

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
# Runtime이 준비될 때까지 폴링
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### AgentCore Runtime 호출

마지막으로 페이로드를 사용하여 AgentCore Runtime을 호출할 수 있습니다.

<div style="text-align:left">
    <img src="images/invoke.png" width=85%"/>
</div>

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "What are some cowboy-themed attractions and museums in Texas?"})
invoke_response

### 호출 결과 처리

이제 호출 결과를 처리하여 애플리케이션에 포함할 수 있습니다.

In [ ]:
from IPython.display import Markdown, display
import json
response_text = invoke_response['response'][0]
display(Markdown(response_text))

### boto3로 AgentCore Runtime 호출

이제 AgentCore Runtime이 생성되었으므로 모든 AWS SDK로 호출할 수 있습니다. 예를 들어 boto3 `invoke_agent_runtime` 메서드를 사용할 수 있습니다.

In [ ]:
import boto3
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

# boto3 클라이언트로 직접 agent 호출
boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "What are some rodeo events happening in Oklahoma?"})
)

# 응답 파싱 및 출력
response_body = boto3_response['response'].read()
response_data = json.loads(response_body)
display(Markdown(response_data.get('result', 'No result found')))

### Amazon CloudWatch의 AgentCore Observability

요약하자면, AgentCore runtime에서 호스팅되는 agent의 observability를 활성화하려면 다음 단계를 따르세요:

- Amazon CloudWatch에서 Transaction Search 활성화
- agent는 trace를 내보내고 opentelemetry 명령을 사용하여 계측됩니다: `opentelemetry-instrument python any_runtime_agent.py`
- requirements.txt 파일에는 Bedrock Agentcore Runtime에 agent를 배포할 때 `aws-opentelemetry-distro`가 나열되어 있습니다.

## GenAI Observability 대시보드의 Bedrock AgentCore 개요

observability가 있는 모든 Agent를 볼 수 있으며 시간 프레임을 기준으로 데이터를 필터링할 수 있습니다.

메인 대시보드에서 모든 agent의 runtime 메트릭을 볼 수 있습니다.

이제 방금 배포한 agent를 클릭하면 이 agent에 특정한 runtime 메트릭 대시보드로 이동하며, 사용자 지정 시간 프레임으로 데이터를 필터링할 수도 있습니다.

Sessions View 탭에서 이 agent와 연결된 모든 세션으로 이동할 수 있습니다.



Trace View 탭에서 runtime에서 이 agent의 trace 및 span 정보를 확인할 수 있습니다.


<div style="text-align:left">
    <img src="images/span_crew_Ai.png" width="60%"/>
</div>


GenAI observability 대시보드의 다양한 기능을 클릭하여 trace에 대한 자세한 정보를 확인하세요.

<div style="text-align:left">
    <img src="images/span_details.png" width="60%"/>
</div>




## 정리 (선택 사항)

이제 생성된 AgentCore Runtime을 정리해 보겠습니다.

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

# AgentCore Runtime 삭제
runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
    
)

# ECR 리포지토리 삭제 (force=True로 이미지 포함 삭제)
response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# 축하합니다!

observability가 활성화된 간단한 CrewAI agent를 Amazon Bedrock AgentCore Runtime에 성공적으로 생성하고 배포했습니다. 이 예제는 다음 방법을 보여줍니다:

- 웹 검색 기능을 갖춘 간단한 CrewAI 여행 agent 생성
- Amazon CloudWatch를 통한 observability 활성화
- SDK 및 boto3를 모두 사용하여 agent 호출

이제 agent를 완전한 observability 및 모니터링 기능과 함께 사용할 수 있습니다.